# Import the libraries

In [1]:
import numpy as np
import os
import glob
import tensorflow as tf
import argparse
import random
from tensorflow.keras.preprocessing import image

import ecdl

import csv
import io
import json

import tensorflow_addons as tfa
import matplotlib.pyplot as plt

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [7]:
class myargs():
    def __init__(self):
        self.exp_id = 'efficientnetb1_round3'
        self.test_data = 'first'
        self.load_type = 'model'
   

args = myargs()


base_exp_path = '../experiments'
exp_path = os.path.join(base_exp_path, args.exp_id)
args_path = os.path.join(exp_path, 'hyperparameters.json')
with open(args_path) as json_file:
    data = json.load(json_file)

model = tf.keras.models.load_model(exp_path)

if args.load_type == 'weight':
    best_metric = 0.0
    chosen_index = 0
    all_weights = glob.glob(os.path.join(exp_path,'*.hdf5'))
    metric_list = [float(weight_file.split('-')[-1].split('.hdf5')[0]) for weight_file in all_weights]    
    for index,metric_value in enumerate(metric_list):
        if best_metric < metric_value:
            best_metric = metric_value
            chosen_index = index
    model.load_weights(all_weights[chosen_index])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(
        num_thresholds=200, curve='ROC', summation_method='interpolation', name=None,
        dtype=None, thresholds=None, multi_label=False, label_weights=None)
        ])    

In [ ]:

parser = argparse.ArgumentParser(description='Choosing the experiment model to load and test')
# parser.add_argument('--exp_id', default='experiments/elbow_efficientnet_lr_1e-4_rot60_ws0.1_hs0.1_zr0.2_round1/weights.16-0.7626.hdf5',type=str,metavar='EXP_ID',help='name of the experiment')
# parser.add_argument('--exp_id', default='experiments/extended_training/weights.25-0.8174.hdf5',type=str,metavar='EXP_ID',help='name of the experiment')
parser.add_argument('--exp_id', default='efficientnetb0_round3',type=str,metavar='EXP_ID',help='name of the experiment')
parser.add_argument('--test_data', default='first',type=str,metavar='EXP_ID',help='Testing Dataset Choice')
parser.add_argument('--load_type', default='model',type=str,metavar='EXP_ID',help='Whether to load model or best weight based on validation result')
args = parser.parse_args()
print(args)


In [ ]:
## Best model
# python scripts/write_results_csv.py --exp_id efficientnetb1_round3 --test_data eval_both --load_type weight